In [1]:
from beanie import Document, Indexed, init_beanie
from motor.motor_asyncio import AsyncIOMotorClient
from pydantic import BaseModel

%load_ext autoreload
%autoreload 2

In [2]:
class Category(BaseModel):
    name: str
    description: str


class Product(Document):
    name: str
    description: str | None = None
    price: Indexed(float)
    category: Category

    class Settings:
        name = "products"


async def init(client):
    await init_beanie(database=client.db_name, document_models=[Product])


In [3]:
client = AsyncIOMotorClient("mongodb://localhost:27017")

In [4]:
client.db_name

AsyncIOMotorDatabase(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=False, driver=DriverInfo(name='Motor', version='3.7.0', platform='asyncio')), 'db_name'))

In [5]:
await client.list_database_names()

['admin', 'config', 'db_name', 'local']

In [6]:
await client.drop_database("db_name")

In [7]:
await init(client)

In [8]:
await Product.get_motor_collection().drop()

In [9]:
chocolate = Category(name="Chocolate", description="A preparation of roasted and ground cacao seeds.")
tonybar = Product(name="Tony's", price=5.95, category=chocolate)
marsbar = Product(name="Mars", price=1, category=chocolate)

In [10]:
await Product.insert_many([tonybar, marsbar])

InsertManyResult([ObjectId('67a4c35e14feebce53b761bc'), ObjectId('67a4c35e14feebce53b761bd')], acknowledged=True)

In [11]:
await Product.find().to_list()

[Product(id=ObjectId('67a4c35e14feebce53b761bc'), revision_id=None, name="Tony's", description=None, price=5.95, category=Category(name='Chocolate', description='A preparation of roasted and ground cacao seeds.')),
 Product(id=ObjectId('67a4c35e14feebce53b761bd'), revision_id=None, name='Mars', description=None, price=1.0, category=Category(name='Chocolate', description='A preparation of roasted and ground cacao seeds.'))]

In [12]:
await Product.find(Product.price > 1).to_list()


[Product(id=ObjectId('67a4c35e14feebce53b761bc'), revision_id=None, name="Tony's", description=None, price=5.95, category=Category(name='Chocolate', description='A preparation of roasted and ground cacao seeds.'))]